In [98]:
import pandas as pd
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt

In [99]:
#df=pd.read_csv("../moreCleanData.csv")
df=pd.read_csv("clean_csv", index_col=0)
#df=df.drop_duplicates()
df.head()

,GENDER,SENIORCITIZEN,PARTNER,DEPENDENTS,TENURE,PHONESERVICE,PAPERLESSBILLING,MONTHLYCHARGES,TOTALCHARGES,MONTHLY_MINUTES_OF_USE,...,STREAMINGMOVIES_No,STREAMINGMOVIES_No internet service,STREAMINGMOVIES_Yes,CONTRACT_Month-to-month,CONTRACT_One year,CONTRACT_Two year,PAYMENTMETHOD_Bank transfer automatic,PAYMENTMETHOD_Credit card automatic,PAYMENTMETHOD_Electronic check,PAYMENTMETHOD_Mailed check
0,0,0,1,0,1,0,1,29.85,29.85,0,...,1,0,0,1,0,0,0,0,1,0
1,1,0,0,0,45,0,0,42.30,1840.75,0,...,1,0,0,0,1,0,1,0,0,0
2,0,0,0,0,10,0,0,29.75,301.90,0,...,1,0,0,1,0,0,0,0,0,1
3,1,1,0,0,1,0,1,39.65,39.65,0,...,0,0,1,1,0,0,0,0,1,0
4,1,0,1,1,1,0,0,30.20,30.20,0,...,1,0,0,1,0,0,0,0,1,0


In [100]:
#Prepare target & features for modeling
cols = df.columns
train_cols = cols.drop(["CHURN"])
features = df[train_cols]
target = df["CHURN"]
train_cols

Index(['GENDER', 'SENIORCITIZEN', 'PARTNER', 'DEPENDENTS', 'TENURE',
       'PHONESERVICE', 'PAPERLESSBILLING', 'MONTHLYCHARGES', 'TOTALCHARGES',
       'MONTHLY_MINUTES_OF_USE', 'TOTAL_MINUTES_OF_USE', 'MONTHLY_SMS',
       'TOTAL_SMS', 'MULTIPLELINES_No', 'MULTIPLELINES_No phone service',
       'MULTIPLELINES_Yes', 'INTERNETSERVICE_DSL',
       'INTERNETSERVICE_Fiber optic', 'INTERNETSERVICE_No',
       'ONLINESECURITY_No', 'ONLINESECURITY_No internet service',
       'ONLINESECURITY_Yes', 'ONLINEBACKUP_No',
       'ONLINEBACKUP_No internet service', 'ONLINEBACKUP_Yes',
       'DEVICEPROTECTION_No', 'DEVICEPROTECTION_No internet service',
       'DEVICEPROTECTION_Yes', 'TECHSUPPORT_No',
       'TECHSUPPORT_No internet service', 'TECHSUPPORT_Yes', 'STREAMINGTV_No',
       'STREAMINGTV_No internet service', 'STREAMINGTV_Yes',
       'STREAMINGMOVIES_No', 'STREAMINGMOVIES_No internet service',
       'STREAMINGMOVIES_Yes', 'CONTRACT_Month-to-month', 'CONTRACT_One year',
       'CONTRAC

## Decision tree

In [101]:
from sklearn import tree
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=1)
clf = tree.DecisionTreeClassifier(class_weight="balanced")
clf = clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

In [102]:
### BEGIN SOLUTION
print(f"Training Data Score: {clf.score(X_train, y_train)}")
print(f"Testing Data Score: {clf.score(X_test, y_test)}")
### END SOLUTION

Training Data Score: 1.0
Testing Data Score: 0.7279954571266326


In [103]:
# False positives.
fp_filter = (predictions == 1) & (y_test == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (y_test == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (y_test == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (y_test == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)
print(tpr)
print(fpr)

0.4988814317673378
0.19406392694063926


In [104]:
### BEGIN SOLUTION
print(f"First 10 Predictions:   {predictions[:10].tolist()}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")
### END SOLUTION
# Model Accuracy
print('Test Acc: %.3f' % clf.score(X_test, y_test))
from sklearn.metrics import roc_curve, auc, roc_auc_score
auc = roc_auc_score(y_test, predictions)
print(f"auc: {auc}")
# Calculate classification report
target_names=['1','0']
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=target_names))

First 10 Predictions:   [0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
First 10 Actual labels: [0, 0, 1, 0, 1, 0, 0, 0, 0, 1]
Test Acc: 0.728
auc: 0.6524087524133493
             precision    recall  f1-score   support

          1       0.83      0.81      0.82      1314
          0       0.47      0.50      0.48       447

avg / total       0.73      0.73      0.73      1761

First 10 Predictions:   [0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
First 10 Actual labels: [0, 0, 1, 0, 1, 0, 0, 0, 0, 1]


## Random forest

In [105]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200,class_weight="balanced")
rf = rf.fit(X_train, y_train)
predictions = rf.predict(X_test)

In [106]:
### BEGIN SOLUTION
print(f"Training Data Score: {rf.score(X_train, y_train)}")
print(f"Testing Data Score: {rf.score(X_test, y_test)}")
### END SOLUTION

Training Data Score: 1.0
Testing Data Score: 0.8006814310051107


In [107]:
# False positives.
fp_filter = (predictions == 1) & (y_test == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (y_test == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (y_test == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (y_test == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)
print(tpr)
print(fpr)

0.4854586129753915
0.09208523592085235


In [108]:
### BEGIN SOLUTION
print(f"First 10 Predictions:   {predictions[:10].tolist()}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")
### END SOLUTION
# Model Accuracy
print('Test Acc: %.3f' % rf.score(X_test, y_test))
from sklearn.metrics import roc_curve, auc, roc_auc_score
auc = roc_auc_score(y_test, predictions)
print(f"auc: {auc}")
# Calculate classification report
target_names=['1','0']
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=target_names))

First 10 Predictions:   [0, 0, 0, 0, 1, 0, 0, 1, 0, 1]
First 10 Actual labels: [0, 0, 1, 0, 1, 0, 0, 0, 0, 1]
Test Acc: 0.801
auc: 0.6966866885272696
             precision    recall  f1-score   support

          1       0.84      0.91      0.87      1314
          0       0.64      0.49      0.55       447

avg / total       0.79      0.80      0.79      1761

